In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json 

In [12]:
def load_data(currency):
    folder = f"data/{currency}"
    all_data = []

    if not os.path.exists(folder):
        return pd.DataFrame()
    
    for file in sorted(os.listdir(folder)):
        if file.endswith(".json"):
            with open(os.path.join(folder, file), "r") as f:
                data = json.load(f)
                try:
                    rate_info = data["exchange_rates"]["exchange_rate"]
                    for rate in rate_info:
                        to_code = rate["@to_currency"]
                        value = float(rate["rate"])
                        date = data["exchange_rates"]["@date"]
                        all_data.append({"date": date, "to": to_code, "rate": value})
                except:
                    pass

    return pd.DataFrame(all_data)

In [13]:
currencies = ["USD", "EUR", "GBP", "JPY", "CNY"]
all_frames = {}

for c in currencies:
    df = load_data(c)
    all_frames[c] = df

In [14]:
for currency, df in all_frames.items():
    if df.empty:
        continue

    top_currencies = df["to"].value_counts().head(3).index

    for to_curr in top_currencies:
        sub = df[df["to"] == to_curr]
        sub["date"] = pd.to_datetime(sub["date"])

        plt.figure(figsize=(10, 4))
        sns.lineplot(data=sub, x="date", y="rate")
        plt.title(f"{currency} to {to_curr} exchange rate over time")
        plt.xlabel("Date")
        plt.ylabel("Exchange Rate")
        plt.show()

In [15]:
for currency, df in all_frames.items():
    if df.empty:
        continue

    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date")
    df["change"] = df.groupby("to")["rate"].diff().abs()

    mean_volatility = df.groupby("to")["change"].mean().sort_values(ascending=False)
    print(f"Volatility for {currency}:")
    print(mean_volatility.head(3))
    print()

###Observations (Quest Book)

1. Some currencies like JPY and CNY show smoother trends.
2. Volatility increased around early 2020, possibly due to COVID-19.
3. USD and EUR are among the most stable base currencies.
4. Major economic events like Brexit in 2016 may show sharp spikes in GBP-related pairs.
5. Exchange rates fluctuate more when base is from developing countries. 